<a href="https://colab.research.google.com/github/jinsunghub/HPC-System-Optimization/blob/main/BigInt_Multiplication_Multithread.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Multithreaded Hello World
- 기본 버전
- mutex로 출력 제어 버전
- lambda 함수 버전

In [ ]:
%%writefile hello_threads.cpp
#include <iostream>
#include <thread>
#include <vector>

using namespace std;

void hello(int id, int num_threads) {
  cout << "Hello from thread " << id << " of " << num_threads << endl;
}

int main(int argc, char** argv) {
  int num_threads = (argc > 1) ? stoi(argv[1]) : 4;
  vector<thread> ts;

  for(int t=0;t<num_threads;t++){
    ts.emplace_back(hello, t, num_threads);
  }
  for(auto &th : ts)th.join();
  return 0;
}

Writing hello_threads.cpp


In [ ]:
!g++ -O2 -std=c++11 hello_threads.cpp -o hello_thread

In [ ]:
!./hello_thread

Hello from thread 0 of 4
Hello from thread Hello from thread 1 of 4
Hello from thread 2 of 4
3 of 4


In [ ]:
!./hello_thread 8

Hello from thread 0 of 8
Hello from thread 3 of 8Hello from thread 1 of 8
Hello from thread 2 of 8

Hello from thread 4 of 8
Hello from thread 5 of 8
Hello from thread 6 of 8
Hello from thread 7 of 8


In [ ]:
%%writefile hello_threads_mutex.cpp
#include <iostream>
#include <thread>
#include <vector>
#include <mutex>

using namespace std;

mutex m;

void hello(int id, int num_threads) {
  lock_guard<mutex> lock_guard(m);
  cout << "Hello from thread " << id << " of " << num_threads << endl;
}

int main(int argc, char** argv) {
  int num_threads = (argc > 1) ? stoi(argv[1]) : 4;
  vector<thread> ts;

  for (int t = 0; t < num_threads; t++) {
    ts.emplace_back(hello, t, num_threads);
  }
  for (auto &th : ts) th.join();
  return 0;
}

Writing hello_threads_mutex.cpp


In [ ]:
!g++ -O2 -std=c++11 hello_threads_mutex.cpp -o hello_thread_mutex

In [ ]:
!./hello_thread_mutex 8

Hello from thread 0 of 8
Hello from thread 1 of 8
Hello from thread 4 of 8
Hello from thread 2 of 8
Hello from thread 5 of 8
Hello from thread 6 of 8
Hello from thread 3 of 8
Hello from thread 7 of 8


In [ ]:
%%writefile hello_threads_lambda.cpp
#include <iostream>
#include <thread>
#include <vector>
#include <mutex>

using namespace std;

mutex m;

int main(int argc, char** argv) {
  int num_threads = (argc > 1) ? stoi(argv[1]) : 4;
  vector<thread> ts;

  auto hello = [&](int id) -> void {
    lock_guard<mutex> lock_guard(m);
    cout << "Hello from thread " << id << " of " << num_threads << endl;
  };
  for (int t = 0; t < num_threads; t++) {
    ts.emplace_back(hello, t);
  }
  for (auto &th : ts) th.join();
  return 0;
}

Writing hello_threads_lambda.cpp


In [ ]:
!g++ -O2 -std=c++11 hello_threads_mutex.cpp -o hello_thread_lambda

In [ ]:
!./hello_thread_lambda 8

Hello from thread 0 of 8
Hello from thread 3 of 8
Hello from thread 1 of 8
Hello from thread 7 of 8
Hello from thread 2 of 8
Hello from thread 5 of 8
Hello from thread 4 of 8
Hello from thread 6 of 8


DMV 분배 전략 비교

In [ ]:
%%writefile dmv.cpp
// Usage_examples:
// !./dmv 1024 1024 8 block              // n_row n_col n_threads dist_method
// !./dmv 1024 1024 8 cyclic             // n_row n_col n_threads dist_method
// !./dmv 1024 1024 8 block_cyclic 64    // n_row n_col n_threads dist_method chunk_size

#include <iostream>
#include <thread>
#include <vector>
#include <cstring>

using namespace std;

void init(vector<double>& A, int m, bool zero) {
  for (int i = 0; i < m; i++) {
    A[i] = zero ? 0 : rand() / double(RAND_MAX);
  }
}

int main(int argc, char** argv) {
  if (argc < 5) {
    cerr << "Missing arguments!" << endl;
  }
  int m = stoi(argv[1]);
  int n = stoi(argv[2]);
  int num_threads = stoi(argv[3]);
  string strategy = argv[4];
  int chunk = (argc > 5) ? stoi(argv[5]) : 1;
  vector<double> A(m*n);
  vector<double> x(n);
  vector<double> b(m);
  init(A, m*n, false);
  init(x, n, false);
  init(b, m, true);
  vector<thread> ts;

  auto start = chrono::high_resolution_clock::now();

  if (strategy == "block") {
    chunk = (m + num_threads - 1) / num_threads;
    auto block = [&](const int id) -> void{
    int lower = id * chunk;
    int upper = min(lower + chunk, m);
    for (int row = lower; row < upper; row++) {
      double acc = 0;
      for (int col = 0; col < n; col++) {
        acc += A[row*n+col] * x[col];
      }
      b[row] = acc;
    }
  };
  for (int t = 0; t < num_threads; t++) {
    ts.emplace_back(block, t);
  }
  for (auto &th : ts) th.join();
  }
  else if (strategy == "cyclic") {
    auto cyclic = [&](const int id) -> void{
      for (int row = id; row < m; row += num_threads) {
        double acc = 0;
        for (int col = 0; col < n; col++) {
          acc += A[row*n+col] * x[col];
      }
      b[row] = acc;
    }
  };
  for (int t = 0; t < num_threads; t++) {
    ts.emplace_back(cyclic, t);
  }
  for (auto &th : ts) th.join();
  }
  else if (strategy == "block_cyclic") {
    auto block_cyclic = [&](const int id) -> void{
      int offset = id * chunk;
      int stride = num_threads * chunk;
      for (int lower = offset; lower < m; lower += stride) {
        int upper = min(lower + chunk, m);
        for (int row = lower; row < upper; row++) {
          double acc = 0;
          for (int col = 0; col < n; col++) {
            acc += A[row*n+col] * x[col];
          }
          b[row] = acc;
    }
  }
};
for (int t = 0; t < num_threads; t++) {
  ts.emplace_back(block_cyclic, t);
}
for (auto &th : ts) th.join();
}
  else {
    cerr << "Unknown strategy!" << endl;
  }

  auto end = chrono::high_resolution_clock::now();
  chrono::duration<double> diff = end - start;
  cout << "Execution time: " << diff.count() << " s" << endl;
  cout << "b[0]: " << b[0] << endl;
  return 0;
}

Writing dmv.cpp


In [ ]:
!g++ -O2 -std=c++11 dmv.cpp -o dmv

In [ ]:
!./dmv 1024 1024 8 block
!./dmv 1024 1024 8 cyclic
!./dmv 1024 1024 8 block_cyclic 64

Execution time: 0.00060295 s
b[0]: 264.403
Execution time: 0.0006404 s
b[0]: 264.403
Execution time: 0.00045744 s
b[0]: 264.403


In [ ]:
print("=== Chunk Size = 1 ===")
!./dmv 1024 1024 8 block_cyclic 1

print("\n=== Chunk Size = 2 ===")
!./dmv 1024 1024 8 block_cyclic 2

print("\n=== Chunk Size = 4 ===")
!./dmv 1024 1024 8 block_cyclic 4

print("\n=== Chunk Size = 8 ===")
!./dmv 1024 1024 8 block_cyclic 8

print("\n=== Chunk Size = 16 ===")
!./dmv 1024 1024 8 block_cyclic 16

print("\n=== Chunk Size = 32 ===")
!./dmv 1024 1024 8 block_cyclic 32

print("\n=== Chunk Size = 64 ===")
!./dmv 1024 1024 8 block_cyclic 64

print("\n=== Chunk Size = 128 ===")
!./dmv 1024 1024 8 block_cyclic 128

=== Chunk Size = 1 ===
Execution time: 0.00060778 s
b[0]: 264.403

=== Chunk Size = 2 ===
Execution time: 0.00043863 s
b[0]: 264.403

=== Chunk Size = 4 ===
Execution time: 0.00045484 s
b[0]: 264.403

=== Chunk Size = 8 ===
Execution time: 0.00055446 s
b[0]: 264.403

=== Chunk Size = 16 ===
Execution time: 0.00047544 s
b[0]: 264.403

=== Chunk Size = 32 ===
Execution time: 0.000553129 s
b[0]: 264.403

=== Chunk Size = 64 ===
Execution time: 0.00051048 s
b[0]: 264.403

=== Chunk Size = 128 ===
Execution time: 0.0004029 s
b[0]: 264.403


Block-Cyclic 분배 전략 성능 분석

실험 조건
Matrix 크기: 1024 x 1024
Thread 수: 8개
Chunk 크기: 1, 2, 4, 8, 16, 32, 64, 128

실험 결과: [실험 결과 표](https://docs.google.com/spreadsheets/d/1FoqE5MGYQmlmZ6zPJf8Q4nfPKDACrPz95uahe9VfxEc/edit?usp=sharing)

분석

최적 성능:

Chunk=4와 Chunk=128이 0.000478초로 가장 빠른 평균 실행 시간을 기록했습니다.
Chunk=8도 최소 시간(0.000392초)에서는 가장 우수한 성능을 보였습니다.


최악 성능:

Chunk=1이 0.000585초로 가장 느린 평균 성능을 보였습니다.
이는 과도하게 작은 chunk가 스레드 간 작업 전환 오버헤드를 증가시키기 때문입니다.


성능 변동성:

Chunk=1과 Chunk=32에서 높은 변동성이 관찰되었습니다 (최대-최소 차이가 큼).
Chunk=4와 Chunk=128은 상대적으로 안정적인 성능을 보였습니다.


흥미로운 패턴:

작은 chunk (4): 세밀한 load balancing으로 스레드 간 작업 분산이 균등합니다.
큰 chunk (128): Block 분배에 가까워 캐시 지역성이 향상되어 좋은 성능을 보입니다.
중간 크기 chunk (8~64): 두 효과의 중간 단계로 일관성 있는 성능을 보였습니다.



결론

Block-cyclic 분배 전략에서 chunk 크기는 성능과 안정성에 중요한 영향을 미칩니다.
본 실험에서는 Chunk=4(세밀한 분산)와 Chunk=128(캐시 지역성)이 최적의
평균 성능을 제공했으며, 애플리케이션 특성에 따라 적절한 chunk 크기를 선택하는
것이 중요함을 확인했습니다.


BigInt Multiplication

In [ ]:
%%writefile bigint_mul.cpp
#include <iostream>
#include <cstring>
#include <cstdint>
#include <cstdio>
#include <cstdlib>
#include <ctime>
#include <chrono>
#include <vector>
#include <thread>
using namespace std;

#define N_LIMBS 16384

struct bigint {
  uint32_t limbs[N_LIMBS];
  bigint(){ memset(limbs, 0, sizeof(limbs)); }
  void randomize(){
    for(int i=0;i<N_LIMBS;i++){
      limbs[i] = rand();
    }
  }
  void print_hex() const{
    for(int i=N_LIMBS-1;i>=0;i--) printf("%08x", limbs[i]);
    printf("\n");
  }
};

struct bigint_long {
  uint32_t limbs[2*N_LIMBS];
  bigint_long(){ memset(limbs, 0, sizeof(limbs)); }
  void print_hex() const{
    for(int i=2*N_LIMBS-1;i>=0;i--) printf("%08x", limbs[i]);
    printf("\n");
  }
};

// Sequential 버전
void bigmul_sequential(const bigint &a, const bigint &b, bigint_long &c){
  memset(c.limbs, 0, sizeof(c.limbs)); // 결과를 저장할 c를 0으로 초기화
  for(int i=0;i<N_LIMBS;i++){
    uint64_t carry = 0;
    for(int j=0;j<N_LIMBS;j++){
      // a.limbs[i]와 b.limbs[j]를 곱해서 c.limbs[i+j]에 저장
      // 32비트 크기의 두 limb를 곱하면 64비트 결과물이 필요하므로, 사전에 64비트로 casting하여 연산
      // 결과를 저장할 c 역시 32비트 array로 이루어져있기 때문에, 연산 결과 64비트 중 상위 32bit는 carry로 처리하여 다음 j iteration에서 처리하도록 함
      // 또한, 하나의 c.limbs[n]은 i+j = n이 되는 모든 i,j 쌍의 결과를 합해야하기 때문에 누적시켜나가야함.
      uint64_t s = (uint64_t)c.limbs[i+j]
                 + (uint64_t)a.limbs[i]*(uint64_t)b.limbs[j]
                 + carry;
      c.limbs[i+j] = (uint32_t)s;
      carry = s >> 32;
    }
    // j = N_LIMBS-1 까지 도달했는데, carry가 남아있다면, 다음 limb인 c.limbs[i+N_LIMBS] 쪽으로 carry 전파
    int k = i + N_LIMBS;
    while(carry && k < 2*N_LIMBS){ // carry 전파 과정에서 carry가 다시 발생하면 가장 상위 limb까지 carry 연쇄 전파
      uint64_t s = (uint64_t)c.limbs[k] + carry;
      c.limbs[k] = (uint32_t)s;
      carry = s >> 32;
      ++k;
    }
  }
}

// Parallel 버전
void bigmul_parallel(const bigint &a, const bigint &b, bigint_long &c, int num_threads)
{
  if(num_threads < 1) num_threads = 1;
  memset(c.limbs, 0, sizeof(c.limbs));

  // 각 스레드가 처리할 i 범위를 block distribution으로 나눔
  int chunk = (N_LIMBS + num_threads - 1) / num_threads;

  vector<thread> threads;
  vector<bigint_long> local_results(num_threads);

  // 각 스레드에서 독립적으로 계산
  auto worker = [&](int thread_id) {
    int i_start = thread_id * chunk;
    int i_end = min(i_start + chunk, N_LIMBS);

    // 이 스레드가 담당하는 i 범위에 대해 계산
    for(int i = i_start; i < i_end; i++){
      uint64_t carry = 0;
      for(int j = 0; j < N_LIMBS; j++){
        uint64_t s = (uint64_t)local_results[thread_id].limbs[i+j]
                   + (uint64_t)a.limbs[i] * (uint64_t)b.limbs[j]
                   + carry;
        local_results[thread_id].limbs[i+j] = (uint32_t)s;
        carry = s >> 32;
      }

      // carry 전파
      int k = i + N_LIMBS;
      while(carry && k < 2*N_LIMBS){
        uint64_t s = (uint64_t)local_results[thread_id].limbs[k] + carry;
        local_results[thread_id].limbs[k] = (uint32_t)s;
        carry = s >> 32;
        ++k;
      }
    }
  };

  // 스레드 생성 및 실행
  for(int t = 0; t < num_threads; t++){
    threads.emplace_back(worker, t);
  }

  // 모든 스레드 완료 대기
  for(auto &th : threads){
    th.join();
  }

  // 각 스레드의 결과를 최종 결과 c에 합치기
  for(int t = 0; t < num_threads; t++){
    uint64_t carry = 0;
    for(int i = 0; i < 2*N_LIMBS; i++){
      uint64_t s = (uint64_t)c.limbs[i]
                 + (uint64_t)local_results[t].limbs[i]
                 + carry;
      c.limbs[i] = (uint32_t)s;
      carry = s >> 32;
    }
  }
}


int main(int argc, char** argv){
  int num_threads = (argc > 1) ? stoi(argv[1]) : 4;
  bigint a, b;
  bigint_long c_seq, c_par;
  a.randomize();
  b.randomize();

  auto t0 = std::chrono::high_resolution_clock::now();
  bigmul_sequential(a, b, c_seq);
  auto t1 = std::chrono::high_resolution_clock::now();

  auto t2 = chrono::high_resolution_clock::now();
  bigmul_parallel(a, b, c_par, num_threads);
  auto t3 = chrono::high_resolution_clock::now();
  // 아래 line을 통해서 입력 a, b와 결과 c를 출력할 수 있음.
  //a.print_hex();
  //b.print_hex();
  //c_par.print_hex();

  // 결과 체크
  bool ok = true;
  for (int i = 0; i < 2*N_LIMBS; i++) {
    if (c_seq.limbs[i] != c_par.limbs[i]) {
      ok = false;
      break;
    }
  }

  double seq_sec = std::chrono::duration<double>(t1 - t0).count();
  double par_sec = std::chrono::duration<double>(t3 - t2).count();
  cout << "[Sequential] " << seq_sec << endl;
  cout << "[Parallel w/ " << num_threads << " threads] " << par_sec << endl;
  cout << "Correctness " << (ok ? "OK" : "MISMATCH") << endl;

  return 0;
}

Writing bigint_mul.cpp


In [ ]:
!g++ -std=c++11 -O2 bigint_mul.cpp -o bigint_mul

In [ ]:
!./bigint_mul 8

[Sequential] 0.170316
[Parallel w/ 8 threads] 0.0225213
Correctness OK


실험 환경

런타임: v5e-1 TPU
N_LIMBS: 16384 (524,288비트 BigInt)
스레드 수: 8

Speedup = Sequential Time / Parallel Time
        = 0.170316 / 0.0225213
        = 7.56

Efficiency = (Speedup / num_threads) × 100%
           = (7.56 / 8) × 100%
           = 94.5%

thread 수 8개 대비 7.56배의 매우 높은 Speedup이 나타났습니다. 이는 작업이 매우 잘 분할되었으며, 병렬화에 따른 오버헤드가 상대적으로 매우 작았음을 의미합니다.

병렬화 전략


1. 순차 실행의 외부 루프는 총 N_LIMBS번 반복되며, 각 반복은 거의 독립적인 곱셈 및 덧셈 작업을 수행합니다. 병렬 버전은 이 외부 루프를 8개의 thread에 Block Distribution 방식으로 균등하게 나누어 할당했습니다.

2. 독립적인 로컬 계산: 각 스레드는 a의 할당된 i 범위에 대해 b 전체를 곱하는 작업을 수행하며, 그 결과를 자신만의 독립적인 배열 local_results[thread_id]에 저장합니다. 이로 인해 작업 중 Race Condition이 발생하지 않습니다.

3. 최소한의 합산 오버헤드: 모든 스레드의 작업이 완료된 후, 메인 스레드에서 순차적으로 8개의 local_results를 최종 결과 c에 단순 덧셈으로 합칩니다.

결론

 큰 정수 곱셈이라는 Bigint_mul 계산 집약적 문제에 대해 효율적인 작업 분배와 낮은 통신 비용을 갖춘 병렬화 전략을 적용하여 거의 선형적인 속도 향상을 달성했습니다.